In [1]:
# Directory

!pwd

/home/ec2-user/SageMaker


In [2]:
# Confirm sagemaker role exists

import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

role = get_execution_role()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='arn:aws:iam::971504885040:role/SageMaker')['Role']['Arn']

In [27]:
# Import files from S3

import pandas as pd
import boto3
import io

bucket = "sagemaker-w210-eth"

# Txt file
data_key = '2021-09-01/transaction_hashes_13136427_13142881.txt'
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=bucket, Key=data_key)
contents = obj['Body'].read()
print(contents.decode("utf-8"))

# CSV file
# file_key = '2021-09-01/tokens_13136427_13142881.csv'
# s3_client = boto3.client('s3')
# obj = s3_client.get_object(Bucket=bucket, Key=file_key)
# df = pd.read_csv(io.BytesIO(obj['Body'].read()))
# df.head()

In [9]:
!pip install awswrangler

import awswrangler as wr

df = wr.s3.read_parquet(path="s3://sagemaker-w210-eth/transactions_small")

In [10]:
df

,transaction_id,block_number,transaction_index,value,gas,gas_price,block_timestamp,max_fee_per_gas,max_priority_fee_per_gas,transaction_type
0,1298866400102,12988664,102,None,21000,28600000000,1628481908,NaN,NaN,0
1,1298866500010,12988665,10,0,94813,57555038094,1628481912,NaN,NaN,0
2,1298866500055,12988665,55,None,42000,35000000000,1628481912,NaN,NaN,0
3,1298866500083,12988665,83,0,46769,32198530871,1628481912,NaN,NaN,0
4,1298866500091,12988665,91,None,228523,31777519047,1628481912,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
1100469,1304551200123,13045512,123,None,21000,76252437559,1629239526,1.310000e+11,5.000000e+09,2
1100470,1304551200141,13045512,141,None,168354,75252437559,1629239526,9.185806e+10,4.000000e+09,2
1100471,1304551200161,13045512,161,0,250000,73000000000,1629239526,NaN,NaN,0
1100472,1304551300008,13045513,8,None,207128,114000000000,1629239547,NaN,NaN,0


In [ ]:
import numpy as np
import pandas as pd

import os
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor    
# from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
print("OK")

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics

from scipy.stats import norm
from scipy import stats

## Create train / dev / test splits

In [ ]:

X_columns = []
y_column = []

X_train = data[data.timestamp<'10/01/2021'][X_columns]
Y_train = data[data.timestamp<'10/01/2021'][Y_column]

X_dev = data[(data.timestamp>='10/01/2021') & (data.timestamp<='11/01/2021')][X_columns]
Y_dev = data[(data.timestamp>='10/01/2021') & (data.timestamp<='11/01/2021')][Y_column]

X_test = data[data.timestamp>'11/01/2021'][X_columns]
Y_test = data[data.timestamp>'11/01/2021'][Y_column]

#create data sets that concatenates X's and y's for EDA
train_data = X_train.copy()
dev_data = X_dev.copy()
test_data = X_test.copy()

## Linear Regression

In [ ]:
# Call in the LinearRegression object
lin_reg = LinearRegression(normalize=True, n_jobs=-1)

lin_reg_features = []

# fit train and test data. 
lin_reg.fit(X_train[lin_reg_features], Y_train)


# Predict dev data. 
y_dev_pred = lin_reg.predict(X_dev)

lr_mse = mean_squared_error(y_dev, y_dev_pred)
lr_rmse = np.sqrt(mean_squared_error(y_dev, y_dev_pred))
lr_r2score = metrics.r2_score(y_dev, y_dev_pred)

print ('The MSE of the Linear Regression model is: %.4f'%lr_mse)
print ('The RMSE of the Linear Regression model is: %.4f'%lr_rmse)
print ('The R^2 of the Linear Regression model is: %.4f'%lr_r2score)


## Ridge Regression

In [ ]:
ridge_reg = Ridge(alpha=0.75, random_state=42)
ridge_reg.fit(X_train[lin_reg_features], Y_train)
rr_train_preds = ridge_reg.predict(X_train[lin_reg_features])
rr_dev_preds = ridge_reg.predict(X_dev[lin_reg_features])

rr_train_mse = mean_squared_error(Y_train, rr_train_preds)
rr_train_rmse = np.sqrt(rr_train_mse)
rr_train_r2score = metrics.r2_score(Y_train, rr_train_preds)
rr_dev_mse = mean_squared_error(y_dev, rr_dev_preds)
rr_dev_rmse = np.sqrt(rr_dev_mse)
rr_dev_r2score = metrics.r2_score(Y_dev, rr_dev_preds)

#print ('The MSE of the train Ridge Regression model is: %.4f'%rr_train_mse)
print ('The RMSE of the train Ridge Regression model is: %.4f'%rr_train_rmse)
print ('The R^2 of the train Ridge Regression model is: %.4f'%rr_train_r2score)
#print ('The MSE of the dev Ridge Regression model is: %.4f'%rr_dev_mse)
print ('The RMSE of the dev Ridge Regression model is: %.4f'%rr_dev_rmse)
print ('The R^2 of the dev Ridge Regression model is: %.4f'%rr_dev_r2score)


## Random Forest

In [ ]:
#Instantiate the random forest
rf = RandomForestRegressor(n_estimators = 1000, max_depth = 3, bootstrap = True, max_features = 4, 
                           min_samples_leaf = 50, random_state = 42)

rf_features = X_train

#Train the model on the training data
rf.fit(X_train[rf_features], y_train)

#Get the predictions on the training data set
rf_train_predictions = rf.predict(X_train[rf_features])

#Get the predictions on the dev data set
rf_dev_predictions = rf.predict(X_dev[rf_features])


rf_MSE_train = mean_squared_error(Y_train, rf_train_predictions)
rf_MSE_dev = mean_squared_error(Y_dev, rf_dev_predictions)
print(rf_MSE_train)
print(rf_MSE_dev)

In [ ]:
rf_feature_importances = rf.feature_importances_
rf_feature_importances_dict = {'Features': rf_features,
                               'Feature_Importance': rf_feature_importances}
rf_feature_importances_df = pd.DataFrame(rf_feature_importances_dict)
rf_feature_importances_df.sort_values(by=['Feature_Importance'], 
                                      ascending=False,
                                      inplace=True)
print(rf_feature_importances_df.head(30))

rf_feature_importances_df = rf_feature_importances_df.set_index('Features')
rf_feature_importances_df['Feature_Importance'].nlargest(15).plot(kind='barh',
                                                                  title='Random Forest Feature Importance')

## Gradient Boosting

In [ ]:
xgb_features = []

#set up the training matrix
dtrain = xgb.DMatrix(X_train[xgb_features], label=Y_train)
xgb_dict = {
        'booster': 'dart',
        #'booster': 'gbtree',
        #'booster': 'gblinear',
        'max_depth': 3,
        'random_state': 42,
        'learning_rate': 0.01,
        'objective': 'reg:squarederror',
        'verbosity': 0,
        'sample_type': 'uniform',
        'subsample': 0.6,
        'normalize_type': 'tree',
        'rate_drop': 0.0,
        #'rate_drop': 0.2,
        'skip_drop': 0.2,
        'min_child_weight': 5,
        
    }
#starting hyperparameter of training rounds
training_rounds = 1000

#train the model
bst = xgb.train(
    xgb_dict,
    dtrain,
    training_rounds,
    evals=[(xgb.DMatrix(X_dev[xgb_features], label=Y_dev), 'RMSE')],
    early_stopping_rounds=10) 


training_rounds = bst.best_ntree_limit
 
#get training predictions and MSE
xgb_train_pred = bst.predict(xgb.DMatrix(X_train[xgb_features]), ntree_limit=training_rounds)
xgb_train_mse = mean_squared_error(xgb_train_pred, Y_train)
xgb_train_rmse = np.sqrt(xgb_train_mse)

#print("XGB train mean squared error:", xgb_train_mse)
print("XGB train root mean squared error:", xgb_train_rmse)

#get dev predictions and MSE
xgb_dev_pred = bst.predict(xgb.DMatrix(X_dev[xgb_features]), ntree_limit=training_rounds)
xgb_dev_mse = mean_squared_error(xgb_dev_pred, Y_dev)
xgb_dev_rmse = np.sqrt(xgb_dev_mse)

#print("XGB dev mean squared error:", xgb_dev_mse)
print("XGB dev root mean squared error:", xgb_dev_rmse)

In [ ]:
#feature importance
xgb_importance_weight = bst.get_score(importance_type='weight')
xgb_importance_gain = bst.get_score(importance_type='gain')
xgb_importance_weight_df = pd.DataFrame.from_dict(xgb_importance_weight,
                                                  orient='index',
                                                  columns=['weight'])
xgb_importance_gain_df = pd.DataFrame.from_dict(xgb_importance_gain,
                                                  orient='index',
                                                  columns=['gain'])
xgb_feature_importance_df = xgb_importance_weight_df.join(xgb_importance_gain_df)
xgb_feature_importance_df.sort_values(by=['weight'], ascending=False, inplace=True)                 
print(xgb_feature_importance_df.head(30))


In [ ]:
plot_importance(bst, max_num_features=30)

## Ensemble

In [ ]:
ensemble_train_preds = (lr_preds_train + rr_train_preds + rf_train_predictions + xgb_train_pred) / 4
print("train_rmse =", np.sqrt(mean_squared_error(ensemble_train_preds, Y_train)))
ensemble_dev_preds = (lr_preds_dev + rf_dev_predictions + xgb_dev_pred) / 3
print("dev_rmse =", np.sqrt(mean_squared_error(ensemble_dev_preds, Y_dev)))
